In [1]:
import gym
import numpy as np
rm='Pong-v4'

In [2]:
%matplotlib inline
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def display_frames_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 144)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    plt.close(anim._fig)
    display(HTML(anim.to_jshtml()))

In [3]:
gym.new_step_api=True
env = gym.make(rm)
# model initialization
H = 400 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
#learning_rate = 1e-3
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):
  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

    # if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    #   print('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

C:\Users\Sagardeep Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\Sagardeep Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
%time hist1 = train_model(env, model, total_episodes=6000)

C:\Users\Sagardeep Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -19.000000. running mean: -19.000000
episode 2.000000, reward total was -20.000000. running mean: -19.010000
episode 3.000000, reward total was -20.000000. running mean: -19.019900
episode 4.000000, reward total was -21.000000. running mean: -19.039701
episode 5.000000, reward total was -21.000000. running mean: -19.059304
episode 6.000000, reward total was -21.000000. running mean: -19.078711
episode 7.000000, reward total was -21.000000. running mean: -19.097924
episode 8.000000, reward total was -20.000000. running mean: -19.106945
episode 9.000000, reward total was -21.000000. running mean: -19.125875
episode 10.000000, reward total was -20.000000. running mean: -19.134616
episode 11.000000, reward total was -21.000000. running mean: -19.153270
episode 12.000000, reward total was -21.000000. running mean: -19.171738
episode 13.000000, reward total was -19.000000. running mean: -19.170020
episode 14.000000, reward total was -20.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -19.969988
episode 115.000000, reward total was -20.000000. running mean: -19.970288
episode 116.000000, reward total was -20.000000. running mean: -19.970585
episode 117.000000, reward total was -21.000000. running mean: -19.980879
episode 118.000000, reward total was -20.000000. running mean: -19.981071
episode 119.000000, reward total was -21.000000. running mean: -19.991260
episode 120.000000, reward total was -21.000000. running mean: -20.001347
episode 121.000000, reward total was -21.000000. running mean: -20.011334
episode 122.000000, reward total was -21.000000. running mean: -20.021221
episode 123.000000, reward total was -20.000000. running mean: -20.021008
episode 124.000000, reward total was -21.000000. running mean: -20.030798
episode 125.000000, reward total was -21.000000. running mean: -20.040490
episode 126.000000, reward total was -20.000000. running mean: -20.040085
episode 127.000000, reward total was -

episode 225.000000, reward total was -19.000000. running mean: -20.293034
episode 226.000000, reward total was -21.000000. running mean: -20.300103
episode 227.000000, reward total was -21.000000. running mean: -20.307102
episode 228.000000, reward total was -21.000000. running mean: -20.314031
episode 229.000000, reward total was -21.000000. running mean: -20.320891
episode 230.000000, reward total was -20.000000. running mean: -20.317682
episode 231.000000, reward total was -20.000000. running mean: -20.314505
episode 232.000000, reward total was -20.000000. running mean: -20.311360
episode 233.000000, reward total was -21.000000. running mean: -20.318247
episode 234.000000, reward total was -21.000000. running mean: -20.325064
episode 235.000000, reward total was -21.000000. running mean: -20.331813
episode 236.000000, reward total was -21.000000. running mean: -20.338495
episode 237.000000, reward total was -21.000000. running mean: -20.345110
episode 238.000000, reward total was -

episode 336.000000, reward total was -21.000000. running mean: -20.450686
episode 337.000000, reward total was -20.000000. running mean: -20.446179
episode 338.000000, reward total was -21.000000. running mean: -20.451718
episode 339.000000, reward total was -20.000000. running mean: -20.447200
episode 340.000000, reward total was -21.000000. running mean: -20.452728
episode 341.000000, reward total was -19.000000. running mean: -20.438201
episode 342.000000, reward total was -20.000000. running mean: -20.433819
episode 343.000000, reward total was -21.000000. running mean: -20.439481
episode 344.000000, reward total was -20.000000. running mean: -20.435086
episode 345.000000, reward total was -21.000000. running mean: -20.440735
episode 346.000000, reward total was -19.000000. running mean: -20.426328
episode 347.000000, reward total was -21.000000. running mean: -20.432065
episode 348.000000, reward total was -21.000000. running mean: -20.437744
episode 349.000000, reward total was -

...

episode 5512.000000, reward total was -19.000000. running mean: -18.110799
episode 5513.000000, reward total was -20.000000. running mean: -18.129691
episode 5514.000000, reward total was -17.000000. running mean: -18.118394
episode 5515.000000, reward total was -17.000000. running mean: -18.107210
episode 5516.000000, reward total was -20.000000. running mean: -18.126138
episode 5517.000000, reward total was -17.000000. running mean: -18.114877
episode 5518.000000, reward total was -19.000000. running mean: -18.123728
episode 5519.000000, reward total was -16.000000. running mean: -18.102491
episode 5520.000000, reward total was -20.000000. running mean: -18.121466
episode 5521.000000, reward total was -17.000000. running mean: -18.110251
episode 5522.000000, reward total was -19.000000. running mean: -18.119149
episode 5523.000000, reward total was -17.000000. running mean: -18.107957
episode 5524.000000, reward total was -21.000000. running mean: -18.136878
episode 5525.000000, rewa

episode 5622.000000, reward total was -21.000000. running mean: -18.090410
episode 5623.000000, reward total was -21.000000. running mean: -18.119506
episode 5624.000000, reward total was -19.000000. running mean: -18.128311
episode 5625.000000, reward total was -17.000000. running mean: -18.117028
episode 5626.000000, reward total was -14.000000. running mean: -18.075858
episode 5627.000000, reward total was -18.000000. running mean: -18.075099
episode 5628.000000, reward total was -16.000000. running mean: -18.054348
episode 5629.000000, reward total was -17.000000. running mean: -18.043805
episode 5630.000000, reward total was -19.000000. running mean: -18.053367
episode 5631.000000, reward total was -17.000000. running mean: -18.042833
episode 5632.000000, reward total was -20.000000. running mean: -18.062405
episode 5633.000000, reward total was -20.000000. running mean: -18.081781
episode 5634.000000, reward total was -18.000000. running mean: -18.080963
episode 5635.000000, rewa

episode 5732.000000, reward total was -18.000000. running mean: -18.277085
episode 5733.000000, reward total was -15.000000. running mean: -18.244314
episode 5734.000000, reward total was -19.000000. running mean: -18.251871
episode 5735.000000, reward total was -13.000000. running mean: -18.199352
episode 5736.000000, reward total was -18.000000. running mean: -18.197358
episode 5737.000000, reward total was -16.000000. running mean: -18.175385
episode 5738.000000, reward total was -15.000000. running mean: -18.143631
episode 5739.000000, reward total was -18.000000. running mean: -18.142195
episode 5740.000000, reward total was -17.000000. running mean: -18.130773
episode 5741.000000, reward total was -20.000000. running mean: -18.149465
episode 5742.000000, reward total was -20.000000. running mean: -18.167970
episode 5743.000000, reward total was -20.000000. running mean: -18.186291
episode 5744.000000, reward total was -17.000000. running mean: -18.174428
episode 5745.000000, rewa

episode 5842.000000, reward total was -17.000000. running mean: -18.238768
episode 5843.000000, reward total was -20.000000. running mean: -18.256380
episode 5844.000000, reward total was -21.000000. running mean: -18.283817
episode 5845.000000, reward total was -16.000000. running mean: -18.260978
episode 5846.000000, reward total was -20.000000. running mean: -18.278369
episode 5847.000000, reward total was -19.000000. running mean: -18.285585
episode 5848.000000, reward total was -14.000000. running mean: -18.242729
episode 5849.000000, reward total was -17.000000. running mean: -18.230302
episode 5850.000000, reward total was -17.000000. running mean: -18.217999
episode 5851.000000, reward total was -17.000000. running mean: -18.205819
episode 5852.000000, reward total was -19.000000. running mean: -18.213761
episode 5853.000000, reward total was -18.000000. running mean: -18.211623
episode 5854.000000, reward total was -17.000000. running mean: -18.199507
episode 5855.000000, rewa

episode 5952.000000, reward total was -19.000000. running mean: -18.028310
episode 5953.000000, reward total was -16.000000. running mean: -18.008027
episode 5954.000000, reward total was -18.000000. running mean: -18.007947
episode 5955.000000, reward total was -17.000000. running mean: -17.997868
episode 5956.000000, reward total was -19.000000. running mean: -18.007889
episode 5957.000000, reward total was -19.000000. running mean: -18.017810
episode 5958.000000, reward total was -19.000000. running mean: -18.027632
episode 5959.000000, reward total was -19.000000. running mean: -18.037356
episode 5960.000000, reward total was -17.000000. running mean: -18.026982
episode 5961.000000, reward total was -17.000000. running mean: -18.016712
episode 5962.000000, reward total was -18.000000. running mean: -18.016545
episode 5963.000000, reward total was -16.000000. running mean: -17.996380
episode 5964.000000, reward total was -18.000000. running mean: -17.996416
episode 5965.000000, rewa